#### Setup Notebook

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [1]:
# Put these at the top of every notebook, to get automatic reloading and inline plotting
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Predicting Price Movements of Cryptocurrencies - Using Convolutional Neural Networks to Classify 2D Images of Chart Data

In [2]:
# This file contains all the main external libs we'll use
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

# For downloading files
from IPython.display import FileLink, FileLinks

# For confusion matrix
from sklearn.metrics import confusion_matrix

from fastai.dataset import read_dir
import pandas as pd

In [4]:
PATH = 'data/btc/btcgraphs_cropped/'

# Data

In [ ]:
!ls {PATH}

In [ ]:
os.listdir(f'{PATH}train')

In [ ]:
files = os.listdir(f'{PATH}train/DOWN')[:5]
files

In [ ]:
img = plt.imread(f'{PATH}train/DOWN/{files[3]}')
print(f'{PATH}train/DOWN/{files[0]}')
print(f'{PATH}train/DOWN/{files[1]}')
plt.imshow(img)
FileLink(f'{PATH}train/DOWN/{files[3]}')

# The Steps to Follow
1. Enable data augmentation, and precompute=True
1. Use `lr_find()` to find highest learning rate where loss is still clearly improving
1. Train last layer from precomputed activations for 1-2 epochs
1. Train last layer with data augmentation (i.e. precompute=False) for 2-3 epochs with cycle_len=1
1. Unfreeze all layers
1. Set earlier layers to 3x-10x lower learning rate than next higher layer
1. Use `lr_find()` again
1. Train full network with cycle_mult=2 until over-fitting

## 0. Setup

In [10]:
arch = resnet34
sz = 480
batch_size = int(64)

## 1. Data Augmentation
**Not using data augmentation this time**

Starting without useing data augmentation because I don't think it makes sense for these graphs, we don't need to generalize to slightly different angles. All plots will always be straight on and square in the frame.

In [7]:
tfms = tfms_from_model(arch, sz)

In [8]:
data = ImageClassifierData.from_paths(PATH, bs=batch_size, tfms=tfms, 
                                      trn_name='train', val_name='valid', test_name='test')

## 2. Choose a Learning Rate

This first pretraining was done with only 500,000 of the 1,000,000 train/valid images.

In [ ]:
learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
learn.save('00_pretrained_480')

In [ ]:
# learn.precompute = True
learn.load('00_pretrained_480')

In [ ]:
lrf = learn.lr_find()

#### Plot learning rate

In [ ]:
learn.sched.plot_lr()

In [ ]:
learn.sched.plot()

In [ ]:
learn.save('01_2_480')

## 3. Train Last Layer

In [ ]:
# learn.precompute = True
learn.load('01_2_480')

In [ ]:
learn.fit(1e-4, 1, cycle_save_name='01_weights')

In [ ]:
learn.save("02_trained_once_480")

TODO

Do some tests on accuracy of training on single epoch

## 4. Train Last Layer with Data Augmentation
**Not actually using any augmentation, this is just a few more rounds of training**

In [ ]:
learn.precompute = True
learn.load("02_trained_once_480")

**TODO**
Load the entire 1,000,000 images.

In [ ]:
# data = ImageClassifierData.from_paths(PATH, bs=batch_size, tfms=tfms, 
#                                       trn_name='train', val_name='valid')#, test_name='test')
# learn = ConvLearner.pretrained(arch, data, precompute=True)

In [ ]:
learn.precompute=False #I don't think this makes a difference without data augmentation

In [ ]:
learn.fit(1e-4, 3, cycle_len=1, cycle_save_name='02_weights')

In [ ]:
learn.save("03_trained_2x_480")

In [ ]:
# learn.precompute = False
learn.load("03_trained_2x_480")

More accuracy test...

## Accuracy Test

In [ ]:
data2 = ImageClassifierData.from_paths(PATH, bs=batch_size, tfms=tfms, 
                                      trn_name='train', val_name='valid', test_name='test')

In [ ]:
learn2 = learn

In [ ]:
learn2.set_data(data2)

In [ ]:
log_preds = learn2.predict(is_test=True)

In [ ]:
ans = pd.read_csv(f'{PATH}test_ans2.csv')

In [ ]:
is_up = ans['up']

In [ ]:
log_preds.shape

In [ ]:
log_preds

In [ ]:
PATH

In [ ]:
lp = pd.DataFrame(log_preds)
lp.to_csv(f'{PATH}log_preds.csv')

In [ ]:
log_preds

## 4.5 
This is where I am trying to pick up to train the whole model.

In [9]:
learn = ConvLearner.pretrained(arch, data, precompute=True)
learn.precompute = False
learn.load("03_trained_2x_480")

In [11]:
data_half = ImageClassifierData.from_paths(PATH, bs=int(batch_size/2/2), tfms=tfms, 
                                      trn_name='train_half', val_name='valid_half', test_name='test')
learn.set_data(data_half)

## 5. Unfreeze Earlier Layers

In [12]:
learn.unfreeze()

## 6. Choose Learning Rate for Early Layers
**3x-10x lower learning rate than next higher layer**

Using a relatively large learning rate to train the previous layers because this data set is not very similar to ImageNet. This is why I chose 3x rather than 10x.

In [13]:
lr = np.array([0.0001/9, 0.0001/3, 0.00001])

## 7. Use `lr_find()` Again

In [14]:
lrf2 = learn.lr_find()
learn.sched.plot_lr()
learn.sched.plot()

HBox(children=(IntProgress(value=0, description='Epoch', max=1), HTML(value='')))

  0%|          | 19/31250 [00:30<13:53:24,  1.60s/it, loss=0.714]

KeyboardInterrupt: 

## 8. Train Full Network

In [ ]:
learn.fit(lr, 3, cycle_len=1, cycle_mult=2, cycle_save_name='03_weights')

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

  0%|          | 6/31250 [00:08<12:49:44,  1.48s/it, loss=0.727] 


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/liam/anaconda3/envs/fastai/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/liam/anaconda3/envs/fastai/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/liam/anaconda3/envs/fastai/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



  0%|          | 45/31250 [01:06<12:46:42,  1.47s/it, loss=0.697]

In [ ]:
learn.save("04_fully_trained_480")

In [ ]:
learn.load("04_fully_trained_480")

# Look at Results

...

In [ ]:
data2 = ImageClassifierData.from_paths(PATH, bs=batch_size, tfms=tfms, 
                                      trn_name='train', val_name='valid', test_name='test')

In [ ]:
learn2 = learn

In [ ]:
learn2.set_data(data2)

In [ ]:
log_preds = learn2.predict(is_test=True)

In [ ]:
ans = pd.read_csv(f'{PATH}test_ans2.csv')

In [ ]:
is_up = ans['up']

In [ ]:
log_preds.shape

In [ ]:
log_preds

In [ ]:
PATH

In [ ]:
lp = pd.DataFrame(log_preds)
lp.to_csv(f'{PATH}log_preds.csv')

In [ ]:
log_preds

# Analyze Results

In [ ]:
data.val_y
log_preds = learn.predict()
preds = np.argmax(log_preds, axis=1)  # from log probabilities to 0 or 1
probs = np.exp(log_preds[:,1])        # pr(dog)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(data.val_y, preds)

In [ ]:
plot_confusion_matrix(cm, data.classes)

In [ ]:
cm

In [ ]:
(cm[0][0]+cm[1][1])/(np.sum(cm))

In [ ]:
np.sum(cm)-(42313)